In [19]:
# library import
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
## 기사 제목을 입력하면, 해당 기사의 키워드와 유사한 내용을 가진 top5 기사 추출
data = pd.read_excel('/content/GPT.xlsx')
data.head(10)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100701e+06,20230406,헤럴드경제,기자없음 기자,"[라이프칼럼] 2050년, 우리의 아이들이 가져야 할 것",IT_과학>IT_과학일반,문화>출판,NaN,NaN,NaN,NaN,이윤진,NaN,"서원대,미국,사회복지학부,한국,신세계","2050년,아이들,이슈,챗GPT,Chat,GPT,가을쯤,버전,사용,사람들,급증,세상...","아이들,챗gpt,스마트폰,인공지능,스마트교육,이윤진,보편화,서원대,모달,상용화,전자...",최근 가장 큰 이슈는 챗GPT(Chat GPT)일 것이다. 작년 가을쯤 첫 버전을 ...,http://biz.heraldcorp.com/view.php?ud=20230406...,NaN
1,2.100201e+06,20230406,머니투데이,김유경|기자,"'AI 교수' 등장 딥브레인AI-인천대, AI 김평원 교수 영상 제작",IT_과학>모바일,IT_과학>콘텐츠,NaN,NaN,NaN,NaN,"멘델,윤석열,김주하,장세영,김평원","프랑스,스페인,중국,러시아","AI 휴먼,은행원,딥브레인AI,AI,메타,교육혁신원장,인천대학교","AI,교수,등장,딥브레인AI,인천대,제작,AI,김평원,교수,영상,인공지능,AI,휴먼...","ai,인공지능,김평원,딥브레인ai,딥브레인,딥러닝,인천대학교,장세영,그간,cb",[머니투데이 김유경 기자] \n인공지능(AI) 휴먼 기반 앵커와 은행원에 이어 대학...,http://news.moneytoday.co.kr/view/mtview.php?n...,NaN
2,7.100501e+06,20230406,전자신문,성현희,"""생성형 AI가 바꾸는 미래는?"" 스마트에프엔, 챗GPT 포럼 개최",IT_과학>모바일,NaN,NaN,NaN,NaN,NaN,"박창선,민병오,김덕진,김태원,박성준","서울,비전포럼","스마트에프엔,스마트에프,대한상공회의소,상명대학교,NIA,한국지능정보사회진흥원","생성,AI,미래,스마트에프엔,개최,포럼,스마트에프엔,인터넷,종합,미디어,스마트,에프...","생성형,챗gpt,스마트에프엔,ai,대한상공회의소,박창선,김덕진,인사이트,비전포럼,김...",인터넷 종합미디어 스마트에프엔은 창간 5주년을 맞아 오는 4월 18일 오전 9시 서...,http://www.etnews.com/20230406000081,NaN
3,7.100501e+06,20230406,전자신문,김명희,"서울과기대, '슈퍼컴퓨터 데이터센터' 개소식",IT_과학>모바일,NaN,NaN,NaN,NaN,NaN,이동훈,NaN,"씨이랩,서울과기대,서울과학기술대,교육부,엔비디아","서울과기대,슈퍼컴퓨터,데이터,센터,개소식,서울과학기술대,슈퍼컴퓨터,데이터,센터,개소...","서울과기대,ai,슈퍼컴퓨터,데이터센터,엔비디아,gpu,이동훈,우유니,그래픽처리장치,...",서울과학기술대가 슈퍼컴퓨터 데이터센터를 개소했다.\n\n서울과기대 슈퍼컴퓨터 데이터...,http://www.etnews.com/20230406000064,NaN
4,2.100601e+06,20230406,한국경제,김주완,영어 중국어 등 50여개 국어에 능통 'AI 교수님' 탄생,사회>교육_시험,IT_과학>모바일,IT_과학>콘텐츠,NaN,NaN,NaN,"장세영,김평원","프랑스,스페인,중국,러시아","딥브레인AI,AI,인천대,메타,인천대학교","영어,중국어,국어,AI,교수,탄생,인공지능,AI,딥브레인AI,전문,기업,딥브레인,A...","ai,인천대,중국어,장세영,딥브레인,김평원,딥브레인ai,인공지능,다국어,50여,러시...",인공지능(AI) 전문 기업 딥브레인AI가 인천대학교와 ‘AI 교수'를 구현했다고 6...,https://www.hankyung.com/economy/article/20230...,NaN
5,2.100801e+06,20230406,아시아경제,김철현,부동산 '발품' 이제 옛말 프롭테크 스타트업 뜬다,경제>부동산,IT_과학>모바일,IT_과학>콘텐츠,NaN,NaN,NaN,"김정수,한아름,안성우",NaN,"직방,자본시장연구원,프롭테크","부동산,발품,옛말,프롭테크,스타트업,뜬다,스타트업,부동산,최신,발품,물건,부동산,업...","부동산,프롭테크,보증금,직방,집품,스타트업,리테일,테넌트,ai,스위트스팟,건축물,빅...","‘발품 팔아야 좋은 물건을 찾을 수 있다’, 부동산 업계의 이 오랜 금언(金言)은 ...",https://view.asiae.co.kr/article/2023040609225...,NaN
6,7.100501e+06,20230406,전자신문,전자신문인터넷 서희원 기자,"헬로앱스, '챗 GPT' 연계형 AI 챗봇 개발 SW 출시",IT_과학>콘텐츠,IT_과학>모바일,NaN,NaN,NaN,NaN,"김영준,가상현",NaN,"진로,Wi,전자신문,헬로앱스,메타","헬로앱스,GPT,챗봇,연계,AI,챗봇,SW,출시,VR,메타,버스,헬로앱스,AI,진로...","vr,챗봇,ai,헬로앱스,사용자,sw,메타버스,진로교육,과학실험,인공지능",VR 메타버스 및 AI 진로체험 교육 전문기업 ㈜헬로앱스(대표 김영준)는 전문적인 ...,http://www.etnews.com/20230405000213,NaN
7,2.100601e+06,20230406,한국경제,서기열,AI 일자리 작년에 미국서만 80만개 생겨 가장 많은 지역은?,국제>미국_북미,경제>취업_창업,IT_과학>인터넷_SNS,NaN,NaN,NaN,NaN,"메사추세츠,일리노이,실리콘밸리,미국,캘리포니아,버지니아,아칸소,플로리다,워싱턴,델라웨어","싱크탱크,구글,중국,텍사스주,AI,워싱턴DC,인간중심AI연구소,메타,뉴욕주,한경 글...","AI,일자리,미국,80만,지역,기사,글로벌마켓,최대,해외,투자,정보,플랫폼,한경,글...","ai,미국,일자리,워싱턴,스탠포드대,델라웨어,캘리포니아,보고서,80만,한경,민간기업...",이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. \...,https://www.hankyung.com/international/article...,NaN
8,2.100851e+06,20230406,아주경제,윤주혜,"구글, AI 슈퍼컴퓨터 공개 ""엔비디아보다 최대 1.7배 빨라""",경제>자동차,IT_과학>인터넷_SNS,경제>반도체,NaN,NaN,NaN,NaN,"최고경영자,장악","젠슨,구글,CNBC,TPU,Un,슈퍼,엔비디아","구글,공개,AI,슈퍼컴퓨터,엔비디아,최대,구글,5일,현지시간,인공지능,AI,v4,슈...","엔비디아,구글,ai,슈퍼컴퓨터,tpu,v4,엠엘퍼프,젠슨,텐서,processing,...",[사진=로이터 연합뉴스] \n \n 구글이 5일(현지시간) 인공지능(AI) 슈퍼컴퓨...,https://www.ajunews.com/view/20230406065320167,NaN
9,2.100501e+06,20230406,파이낸셜뉴스,홍창기 기자 (theveryfirst@fnnews.com),메타버스에서 AI로 '빠꾸'한 저커버그...경영진과 함께 AI에 초집중 왜?,IT_과학>모바일,경제>자동차,IT_과학>콘텐츠,NaN,NaN,NaN,"크리스,저커버그,콕스,앤드류,마크 저커버그","보스워스,최고경영자,실리콘밸리,미국,일본,멜로파크,LLaMa","구글,니케이,MS,삼성전자,페이스북,보스워스,CNBC,마이크로소프트,애플,최고제품책...","메타버스,AI,빠꾸,경영진,AI,초집중,메타버스,분야,순손실,눈덩이,AI,경쟁,경쟁...","메타,ai,메타버스,저커버그,미국,경영진,실리콘밸리,보스워스,구글,페이스북,책임자,...",저커버그 올인 메타버스 분야 순손실 18조로 눈덩이로 불어나고 \nAI 경쟁에서 경...,http://www.fnnews.com/news/202304060649115833,NaN


In [22]:
## Null값 확인
print('키워드 열의 결측값의 수:',data['키워드'].isnull().sum())

키워드 열의 결측값의 수: 0


In [23]:
## 키워드열에 대해 TF-IDF 행렬을 구하기
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['키워드'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape) # 문서 수, 단어 수

TF-IDF 행렬의 크기(shape) : (4082, 47425)


In [24]:
# 4082 문서 벡터에 대해 상호간의 코사인 유사도 구하기
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (4082, 4082)


In [27]:
# 기사 제목 = key, 기사의 인덱스 = value
title_to_index = dict(zip(data['제목'], data.index))
idx = title_to_index['[라이프칼럼] 2050년, 우리의 아이들이 가져야 할 것']
print(idx)

0


In [31]:
# 선택한 기사 제목을 입력하면 코사인 유사도를 통해 가장 키워드가 유사한 5개의 기사를 찾아냄
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 기사의 인덱스
    idx = title_to_index[title]

    # 해당 기사와 모든 기사와의 유사도
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    # 가장 유사한 5개 기사의 인덱스 
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 5개 기사의 제목 return .
    return data['제목'].iloc[movie_indices]

In [35]:
get_recommendations('구글, AI 슈퍼컴퓨터 공개 "엔비디아보다 최대 1.7배 빨라"')

2758                       챗GPT에 "너의 뇌세포는?" 반도체 업계가 안은 숙제
719                 "누구나 생성형AI 만들 수 있다" 엔비디아 '슈퍼컴 구독' 서비스
3029                 엔비디아는 '챗GPT 시대 황태자' 올 들어 시총 208兆 불어나
1550                        "엔비디아, 고성능 반도체로 AI붐 중심에" CNBC
775     MS 구글도 줄서는 엔비디아 신기술 “아이폰 같은 파급력” 예고 [정혜진의 Whyn...
Name: 제목, dtype: object